Установим необходимые пакеты для

In [1]:
pip install redis


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Подключимся и проверим, что все работает корректно

In [2]:
import redis
pool = redis.ConnectionPool(host='172.17.0.2', port=6379, db=0)
r = redis.Redis(connection_pool=pool)

In [3]:
r.flushdb()
r.set('foo', 'bar')
# True
r.get('foo')
# bar

b'bar'

Теперь выкачаем датасет для тестов

In [4]:
!rm large-file.json
!wget https://raw.githubusercontent.com/json-iterator/test-data/master/large-file.json

--2023-04-26 02:53:33--  https://raw.githubusercontent.com/json-iterator/test-data/master/large-file.json
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26141343 (25M) [text/plain]
Saving to: ‘large-file.json’

large-file.json     100%[===================>]  24.93M  1.20MB/s    in 23s     

2023-04-26 02:53:57 (1.10 MB/s) - ‘large-file.json’ saved [26141343/26141343]



In [5]:
import json

In [6]:
f = open('large-file.json')

data = json.load(f)

for i in data[:5]:
    print(i)
f.close()

{'id': '2489651045', 'type': 'CreateEvent', 'actor': {'id': 665991, 'login': 'petroav', 'gravatar_id': '', 'url': 'https://api.github.com/users/petroav', 'avatar_url': 'https://avatars.githubusercontent.com/u/665991?'}, 'repo': {'id': 28688495, 'name': 'petroav/6.828', 'url': 'https://api.github.com/repos/petroav/6.828'}, 'payload': {'ref': 'master', 'ref_type': 'branch', 'master_branch': 'master', 'description': "Solution to homework and assignments from MIT's 6.828 (Operating Systems Engineering). Done in my spare time.", 'pusher_type': 'user'}, 'public': True, 'created_at': '2015-01-01T15:00:00Z'}
{'id': '2489651051', 'type': 'PushEvent', 'actor': {'id': 3854017, 'login': 'rspt', 'gravatar_id': '', 'url': 'https://api.github.com/users/rspt', 'avatar_url': 'https://avatars.githubusercontent.com/u/3854017?'}, 'repo': {'id': 28671719, 'name': 'rspt/rspt-theme', 'url': 'https://api.github.com/repos/rspt/rspt-theme'}, 'payload': {'push_id': 536863970, 'size': 1, 'distinct_size': 1, 'ref'

In [7]:
def measure_time(conn, func):
    start = r.time()
    func()
    end = r.time()
    seconds = end[0] - start[0]
    microseconds = end[1] - start[1]
    print('Microseconds: ', seconds * 1000000 + microseconds, '\n')

Загружаем как строки

In [8]:
def load_as_strings(data, db):
    id = 0 
    for curr in data:
        db.set('event:' + str(id), str(curr))
        id += 1

measure_time(r, lambda : load_as_strings(data, r))


Microseconds:  1070992 



Читаем как строки

In [9]:
def read_as_strings(db, size):
    for id in range(size):
        db.get('event:' + str(id))


measure_time(r, lambda : read_as_strings(r, len(data)))

Microseconds:  722833 



Создадим hash map id - весь объект

In [10]:
def load_hash_map(db, data):
    for curr in data:
        db.hset('info_by_id', str(curr['id']), str(curr))

measure_time(r, lambda : load_hash_map(r, data))

Microseconds:  957494 



Прочитаем

In [11]:
def read_hash_map(db, data):
    for curr in data:
        db.hget('info_by_id', str(curr['id']))

measure_time(r, lambda : read_hash_map(r, data))

Microseconds:  666599 



Абсолютно то же самое, только для zset

In [12]:
def load_zhash_map(db, data):
    for curr in data:
        dict = {}
        dict[str(curr)] = int(curr['id'])
        db.zadd('zinfo_by_id', dict)

measure_time(r, lambda : load_zhash_map(r, data))

Microseconds:  1139101 



In [13]:
def read_zhash_map(db):
    db.zrange('zinfo_by_id', 0, -1)

measure_time(r, lambda : read_zhash_map(r))

Microseconds:  88352 



И последнее - лист

In [14]:
def load_list(db, data):
    for curr in data:
        db.lpush('list_of_everything',str(curr))

measure_time(r, lambda : load_list(r, data))

Microseconds:  997057 



In [15]:
def read_list(db):
    db.lrange('list_of_everything', 0, -1)

measure_time(r, lambda : read_list(r))

Microseconds:  52271 

